In [36]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
llm("explain large language models in one sentence")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nLarge language models are advanced natural language processing systems that use vast amounts of data and complex algorithms to generate human-like text and understand language in a variety of contexts.'

In [3]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [4]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data")
]
response = chat(messages)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [5]:
print(response.content, end='\n')

Sure! Here is an example Python script that trains a simple neural network on simulated data using the TensorFlow library:

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 2)
y = np.array([1 if x[0] + x[1] > 1 else 0 for x in X])

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')
```

This script generates 1000 data points with 2 features each and corresponding binary labels. It then defines a simple neural network with one hidden layer and trains it on the simulated data for 10 epochs. Finally, it ev

In [6]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [7]:
prompt

PromptTemplate(input_variables=['concept'], template='\nYou are an expert data scientist with an expertise in building deep learning models.\nExplain the concept of {concept} in a couple of lines\n')

In [8]:
llm(prompt.format(concept="regularization"))

'\nRegularization is a technique used in deep learning to prevent overfitting of the model on the training data. It involves adding a penalty term to the loss function, which penalizes complex models and encourages simpler ones, thus preventing them from memorizing the training data and improving their generalization ability.'

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.invoke("autoencoder"))

{'concept': 'autoencoder', 'text': 'Autoencoder is a type of neural network that learns to compress and reconstruct its input data. It consists of an encoder that compresses the data into a lower-dimensional representation and a decoder that reconstructs the original data from the compressed representation. This unsupervised learning technique is commonly used for data compression, denoising, and anomaly detection.'}


In [10]:
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [11]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.invoke("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of neural network that learns to compress and reconstruct data by passing it through a bottleneck layer. It uses unsupervised learning to train itself on the input data, and can be used for dimensionality reduction, data denoising, and feature extraction tasks. The goal of an autoencoder is to minimize the reconstruction error between the original input and the reconstructed output. 


An autoencoder is like a special brain that learns how to shrink and then make things bigger again. It looks at pictures and figures out how to make them smaller, and then how to make them big again without losing any important parts. It's really good at this because it practices a lot without anyone telling it what to do. It can also help us make better pictures or find hidden things in the pictures. The best part is, it's always trying to make the pictures look exactly the same as the original!

> Finished chain.
{'input': 'autoen

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
)

explanation_text = explanation.get("output", "")  # Replace "text" with the key containing the text
texts = text_splitter.create_documents([explanation_text])

In [13]:
texts

[Document(page_content='An autoencoder is like a special brain that learns how to shrink and then make things bigger again.'),
 Document(page_content='It looks at pictures and figures out how to make them smaller, and then how to make them big again'),
 Document(page_content="without losing any important parts. It's really good at this because it practices a lot without"),
 Document(page_content='anyone telling it what to do. It can also help us make better pictures or find hidden things in the'),
 Document(page_content="pictures. The best part is, it's always trying to make the pictures look exactly the same as the"),
 Document(page_content='original!')]

In [14]:
texts[0].page_content

'An autoencoder is like a special brain that learns how to shrink and then make things bigger again.'

In [15]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:
query_result = embeddings.embed_query(texts[0].page_content)
query_result

[-0.027068273425135975,
 0.00236912228817672,
 0.025836720422648972,
 -0.0008766724278053337,
 -7.013582043305291e-05,
 0.0019542828183806067,
 0.002852021180225301,
 -0.013754520224552917,
 -0.016710251155811928,
 -0.047576899393039204,
 0.013171152816780643,
 0.04322108519301065,
 0.02008082295385088,
 0.0033576068567988883,
 0.013099851693556883,
 -0.009826509969899266,
 0.011809960646441083,
 0.008206043189144793,
 -0.0065499258467784415,
 -0.023658813322634697,
 -0.02914247217110037,
 0.022660605048082483,
 -0.008102332972449808,
 -0.01711212801267436,
 -0.008432908706088479,
 -0.004028479934530535,
 0.02259578686114003,
 -0.02201241759072265,
 0.01970487411682347,
 -0.009541307339649332,
 0.027742388902330828,
 -0.0012939426494609804,
 -0.0021147089486502372,
 -0.03998015349414681,
 -0.013741557145957957,
 -0.01016356677717414,
 0.01470087339075764,
 -0.022738387244942444,
 0.008478280878154665,
 0.008504208897989685,
 0.010617296879738959,
 0.015763898920929758,
 -0.009353332921